In [1]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import os
import gradio as gr

In [ ]:
#codigo para usar google drive , los modelos estaran guardados en drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
from diffusers import StableDiffusionXLPipeline
from safetensors.torch import load_file
import torch
import os
from PIL import Image
import gradio as gr

# Montar Google Drive
drive.mount('/content/drive')

# Ruta donde se guardarán las imágenes generadas
OUTPUT_DIR = "/content/drive/MyDrive/generated_images"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Función para obtener archivos .safetensors desde Google Drive
def obtener_archivos_safetensor(ruta):
    try:
        return [f for f in os.listdir(ruta) if f.endswith('.safetensors')]
    except FileNotFoundError:
        return []

# Ruta de modelos y LoRA en Google Drive
MODELOS_PATH = "/content/drive/MyDrive"
modelos_disponibles = obtener_archivos_safetensor(MODELOS_PATH)

# Inicializar el pipeline
def cargar_pipeline(model_path):
    return StableDiffusionXLPipeline.from_single_file(
        model_path,
        torch_dtype=torch.float16
    ).to("cuda")

# Backend para procesar inputs y generar imágenes
def generar_imagenes(
    modelo, lora_1, lora_1_weight, lora_2, lora_2_weight,
    prompt, prompt_negativo, guidance_scale, cantidad_imagenes
):
    # Cargar el pipeline con el modelo seleccionado
    pipeline = cargar_pipeline(os.path.join(MODELOS_PATH, modelo))

    # Cargar los LoRA seleccionados
    pipeline.load_lora_weights(os.path.join(MODELOS_PATH, lora_1), adapter_name="lora_1")
    pipeline.load_lora_weights(os.path.join(MODELOS_PATH, lora_2), adapter_name="lora_2")
    pipeline.set_adapters(["lora_1", "lora_2"], adapter_weights=[lora_1_weight, lora_2_weight])

    # Generar las imágenes
    imagenes_generadas = []
    for i in range(cantidad_imagenes):
        image = pipeline(
            prompt=prompt,
            negative_prompt=prompt_negativo,
            guidance_scale=guidance_scale
        ).images[0]

        # Guardar la imagen en Google Drive
        save_path = os.path.join(OUTPUT_DIR, f"generated_image_{i + 1}.png")
        image.save(save_path)
        imagenes_generadas.append(image)

    return imagenes_generadas

# Configurar la interfaz de usuario en Gradio
interfaz = gr.Interface(
    fn=generar_imagenes,
    inputs=[
        gr.Dropdown(choices=modelos_disponibles, label="Modelo"),
        gr.Dropdown(choices=modelos_disponibles, label="LoRA 1"),
        gr.Number(label="Peso LoRA 1 (0-2)", value=1.0, minimum=0, maximum=2, step=0.1),
        gr.Dropdown(choices=modelos_disponibles, label="LoRA 2"),
        gr.Number(label="Peso LoRA 2 (0-2)", value=1.0, minimum=0, maximum=2, step=0.1),
        gr.Textbox(label="Prompt", placeholder="Describe la imagen que deseas generar"),
        gr.Textbox(label="Prompt Negativo", placeholder="Describe lo que no deseas en la imagen"),
        gr.Number(label="Guidance Scale", value=7.5, minimum=0, step=0.1),
        gr.Number(label="Cantidad de Imágenes (1-5)", value=1, minimum=1, maximum=5, step=1)
    ],
    outputs=[
        gr.Gallery(label="Imágenes Generadas"),  # Galería sin estilo adicional
    ],
    title="Generador de Imágenes con LoRA",
    description="Selecciona un modelo, configura los LoRA y otros parámetros para generar imágenes. Las imágenes se guardarán automáticamente en Google Drive."
)

# Lanzar la interfaz
interfaz.launch()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Mounted at /content/drive
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1965e1ec6bee69501f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
